# CLAHE augmentations

You can look into my other notebooks to get more detailed augmentation insights.
list of my other notebooks - 

1. Understanding Image Augmentation - [link](https://www.kaggle.com/amritpal333/understanding-image-augmentation-ranzcr-comp)
2. CLAHE augmentation  - [link](https://www.kaggle.com/amritpal333/clahe-augmentation-ranzcr-comp)
3. Findings your own mean,standerd deviation for the dataset - [link](https://www.kaggle.com/amritpal333/using-custom-mean-std-ranzcr-comp)


## Upvote the notebooks if you find them insightful.

# What is CLAHE?

**Contrast Limited Adaptive Histogram Equalization (CLAHE)** to equalize images. 

CLAHE is a variant of Adaptive histogram equalization (AHE) which takes care of over-amplification of the contrast. 
Operates on small regions in the image, called tiles, rather than the entire image. 
The neighboring tiles are then combined using bilinear interpolation to remove the artificial boundaries.

### This algorithm can be applied to improve the contrast of images.



![CLAHE](https://docs.opencv.org/master/histogram_equalization.png)

In [ ]:
#!pip install albumentations
import albumentations

import torch
from albumentations import ( Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, IAAAdditiveGaussianNoise, Transpose, ToGray )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import matplotlib.pyplot as plt

seed = 42

import pandas as pd
import os
import cv2
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm

Albumentation module
https://albumentations.ai/docs/api_reference/augmentations/transforms

In [ ]:
train_path = '../input/ranzcr-clip-catheter-line-classification/train'
train_files = os.listdir(train_path)

train_df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

train = train_df.reset_index(drop=True) # reset index on both dataframes

print(train.shape)

num_channel = 3

img_size = 255

# Baseline

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		#image = image*(1/recale_image_by)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:

def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 9 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(25, 25)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(3,3, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            

In [ ]:
mean_list = [[ 0.4914168 , 0.4914168 , 0.4914168] , [0.485,0.456,0.406] , [0.9,0.9,0.9] , 0.496 ]
std_list = [[0.407278, 0.407278 , 0.407278] , [0.229,0.224,0.225] , [0.9,0.9,0.9] , 0.407278]

mean = mean_list[0]
std=  std_list[0]

print(mean , std)

CLAHE function

> clip_limit - upper threshold value for contrast limiting. If clip_limit is a single float value, the range will be (1, clip_limit). Default: (1, 4).

> tile_grid_size  - size of grid for histogram equalization. Default: (8, 8).
 
> p  	- probability of applying the transform. Default: 0.5.
 
Targets: image

# Reference images

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                     #albumentations.Normalize(mean= mean ,std= std ,),
                                    ])

Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.CLAHE(clip_limit=(1,4), p= 1),
                                        #albumentations.Normalize(mean= mean ,std= std ,)
                                    ])
Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.CLAHE(clip_limit=(1,10), p= 1),
                                                                            ])
Show_Xrays(train_augs)

## How far can we go?

setting clim limit to 1,40
and tile_grid_size to 15,15

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.CLAHE(clip_limit=(1,30), tile_grid_size = (10,10) ,p= 1),
                                   ])
Show_Xrays(train_augs)

These look good to use.

# Trying differnt combinations of image augmentation

(I hope to find one that magically works!)

In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                #albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.3), lightness=(0.5, 0.7), p=1),
                #albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),
				#albumentations.Normalize(mean= mean ,  std= std ,) 
               ])

Show_Xrays(train_augs)



In [ ]:
mean = [0.485,0.456,0.406]
std = [0.229,0.224,0.225]

transform = albumentations.Compose([  albumentations.RandomCrop(width=256, height=256),
    albumentations.Normalize(mean= mean ,  std= std ,) 
])

# Read an image with OpenCV and convert it to the RGB colorspace
image = cv2.imread("../input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.10003659706701445041816900371598078663.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Augment an image
transformed = transform(image=image)
transformed_image = transformed["image"]


fig = plt.figure()
fig.set_size_inches(15, 15)

ax = plt.subplot(1,1, 1)
plt.tight_layout()
ax.axis('off')
plt.imshow(image)